# **Reducing Traffic Mortality in the USA**

Note that the data used for this report is from the year 2022

By **Mike Musas**    |    *August 10, 2024*

## **1. Introduction**

Traffic mortality has been a public health issue in the United States, with thousands of individuals dying each year due to various factors contributing to road accidents. This project aims to identify the leading causes of traffic fatalities and recommend effective strategies to reduce the number of deaths on American roads.

The project will examine the leading contributors to traffic fatalities, including speeding, drunk driving, and distracted driving. Speeding remains one of the most significant factors, as higher speeds reduce the driver’s ability to react to sudden changes and increase the severity of accidents. Drunk driving is another major cause, with impaired judgment and delayed reaction times leading to a higher likelihood of crashes. Distracted driving, often due to mobile phone use or other in-vehicle distractions, has also emerged as a critical concern in recent years.

**The project also aims to answer the following questions:**

* How has the total number of traffic fatalities changed over the past years?
* What are the age and gender distributions of traffic mortality victims?
* How does traffic mortality vary by state or region?
* Do the number of fatalities correlates with the population of regions?
* How does the fatality rate varies between rural and urban areas?
* What are the leading causes of traffic fatalities in the USA in 2022?

To answer all these questions, we are making use of Tableau desktop (for data visualization and also to create our report) and Bigquery (to create our database and store all the tables we are using).

## **2. About the Data**

### **2.a Database and Source**

The data used to answer the project’s questions come from the [National Highway Traffic Safety Administration](https://www.nhtsa.gov/file-downloads?p=nhtsa/downloads/FARS/) or NHTSA website. The data is comprised of 33 tables. However, 8 tables were used to answer our questions. Before doing our analysis, the data had to go through a cleaning process where the necessary number of columns were selected from every table.

### **2.b Entity-Relationship Diagram (ERD)**
 
The image below shows the entity-relationship diagram or ERD of the database used in this project. The ERD picture the relationship below all the tables in the database. This helps us understand the structure of the database and also understand how we can link different tables together. Note that the ERD below only shows a couple of columns for some of the tables. The reason behind this is to make sure everything is visible on the image.


<img src="Assets/msba_capston_diagram.png" alt="drawing" width="1000"/>

<p>.</p>

## **3. Exploratory Data Analysis**

### **3.a There has been an Upward Trend in the total fatalities on the U.S.A. from 2010 to 2022**

Although the number of fatalities slightly decreased (**-1.68%**) in 2022 compared to 2021, the total number of death due to road traffic is huge and needs to be addressed as soon as possible. There were a total of **42,514 individuals who died in 2022**.

<img src="Assets/Total Fatalities Over the Years, from 2010 to 2022.png" alt="drawing" width="1000"/>

<p>.</p>

In 2010, the total number of road fatalities stood at 32,999. This figure remained relatively stable over the next few years, with minor fluctuations observed until 2014. This stability suggests that existing road safety measures during this period were relatively effective in maintaining a consistent number of fatalities.

Starting from 2015, there is a noticeable and steady increase in the number of road fatalities. This upward trend continues, reaching its peak in 2021 with a total of 43,230 fatalities. The increase during this period raises significant concerns about the effectiveness of road safety policies and the potential impact of other factors such as increased vehicle usage, distracted driving, or infrastructure issues.

In 2022, there is a slight decrease in road fatalities to 42,514, down from the peak observed in the previous year. While this reduction is a positive sign, the overall number of fatalities remains high, indicating that further measures are necessary to achieve a more substantial and sustained decrease.

The trend in US road fatalities from 2010 to 2022 highlights both periods of stability and significant increases, culminating in a peak in 2021. Despite a slight reduction in 2022, the overall trend underscores the critical need for comprehensive and proactive approaches to road safety to protect lives and reduce the number of fatalities on US roads.

### **3.b How do traffic fatalities vary by demographic factors such as age, gender, and location?**



In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
db_name = 'nhtsa'
db_host = 'localhost'
db_username = 'root'
db_password = 'tennisset10'


# Replace these variables with your database configuration
db_type = 'mysql+pymysql'
username = 'root'
password = 'tennisset10'
host = 'localhost'
port = '3306'
database = 'nhtsa'

# Create the database engine
engine = create_engine(f'{db_type}://{username}:{password}@{host}:{port}/{database}')

#### **Fatalities by Gender**

In [11]:
fatalities_by_gender = pd.read_sql_query("""
    WITH fatalities_by_gender AS (
        SELECT
            p.sex,
            COUNT(*) AS total_fatalities
        FROM nhtsa.accidents AS a
        INNER JOIN nhtsa.persons AS p
            ON a.st_case = p.st_case
        WHERE p.died_at_scene_en_route IN ('Died at Scene', 'Died En Route')
            AND p.sex IN ('Male', 'Female')
            AND p.age < 120
        GROUP BY p.sex
    )
    SELECT
        sex,
        total_fatalities,
        total_fatalities / (SELECT SUM(total_fatalities) FROM fatalities_by_gender) * 100 AS percentage_of_total_fatalities
    FROM fatalities_by_gender;
""", engine)

In [12]:
fatalities_by_gender

,sex,total_fatalities,percentage_of_total_fatalities
0,Male,17909,73.2504
1,Female,6540,26.7496


# **73.25%**
of people who died in a car accident in 2022 in the USA were **males** compared to only 26.75% of females.

#### Fatalities by Urban and Rural

In [13]:
fatalities_urban_rural = pd.read_sql_query("""
    SELECT
        rural_urban,
        SUM(fatals) AS total_fatalities,
        SUM(fatals) / (SELECT SUM(fatals) FROM nhtsa.accidents WHERE rural_urban IS NOT NULL) * 100 AS percentage_of_total_fatalities
    FROM nhtsa.accidents AS a
    WHERE rural_urban IS NOT NULL
    GROUP BY rural_urban;
""", engine)

In [14]:
fatalities_urban_rural

,rural_urban,total_fatalities,percentage_of_total_fatalities
0,Rural,17283.0,40.852361
1,Urban,25023.0,59.147639


# **59.15%**
of fatalities on US roads happened in **urban** areas compared to 40.85% in rural areas

#### **Fatalities by Age Group**

In [15]:
fatalities_by_age = pd.read_sql_query(
    '''
    SELECT
        CASE
            WHEN p.age >= 0 AND p.age < 12 THEN '0 >= Age < 12'
            WHEN p.age >= 12 AND p.age < 24 THEN '12 >= Age < 24'
            WHEN p.age >= 24 AND p.age < 36 THEN '24 >= Age < 36'
            WHEN p.age >= 36 AND p.age < 48 THEN '36 >= Age < 48'
            WHEN p.age >= 48 AND p.age < 60 THEN '48 >= Age < 60'
            WHEN p.age >= 60 AND p.age < 72 THEN '60 >= Age < 72'
            WHEN p.age >= 72 AND p.age < 84 THEN '72 >= Age < 84'
            WHEN p.age >= 84 AND p.age < 96 THEN '84 >= Age < 96'
            WHEN p.age >= 96 AND p.age < 108 THEN '96 >= Age < 108'
            ELSE 'Unknown Age'
        END AS age_range,
        COUNT(*) AS total_fatalities
    FROM accidents AS a
    INNER JOIN persons AS p
        ON a.st_case = p.st_case
    WHERE
        p.died_at_scene_en_route IN ('Died at Scene', 'Died En Route')
        AND p.age < 120
    GROUP BY
        CASE
            WHEN p.age >= 0 AND p.age < 12 THEN '0 >= Age < 12'
            WHEN p.age >= 12 AND p.age < 24 THEN '12 >= Age < 24'
            WHEN p.age >= 24 AND p.age < 36 THEN '24 >= Age < 36'
            WHEN p.age >= 36 AND p.age < 48 THEN '36 >= Age < 48'
            WHEN p.age >= 48 AND p.age < 60 THEN '48 >= Age < 60'
            WHEN p.age >= 60 AND p.age < 72 THEN '60 >= Age < 72'
            WHEN p.age >= 72 AND p.age < 84 THEN '72 >= Age < 84'
            WHEN p.age >= 84 AND p.age < 96 THEN '84 >= Age < 96'
            WHEN p.age >= 96 AND p.age < 108 THEN '96 >= Age < 108'
            ELSE 'Unknown Age'
        END
    ORDER BY total_fatalities DESC;
''', engine)

In [16]:
fatalities_by_age

,age_range,total_fatalities
0,24 >= Age < 36,6265
1,36 >= Age < 48,4854
2,48 >= Age < 60,4106
3,12 >= Age < 24,4005
4,60 >= Age < 72,3147
5,72 >= Age < 84,1410
6,84 >= Age < 96,348
7,0 >= Age < 12,325
8,96 >= Age < 108,16


<img src="Assets/Total Number of Fatalities by Age Groups.png" alt="drawing" width="1000"/>

<p>.</p>

The distribution of traffic fatalities across different age ranges shows that certain age groups are more vulnerable than others. **The data reveals tha the highest number of fatalities occurs in the age group 24 to 36, followed by 12 to 24 and 36 to 48**. It can also be seen that these are groups are made of young individuals. These age groups should be prioritized for targeted interventions.

Young people are more prompt to be in front of the car wheel compare to other groups. This is also one reason the top three groups are young. We should make sure that our young population is well informed and educated about the risks and consequencies of not following traffic laws. 

The group with the smallest number of fatalities were olders and children. A total of 1,602 helder people died in 2022 due on the road were 84 years old or older, they are making 3.77% of the total fatalities while children under the age of 12 years old or below made 10.41%. Most individuals tend to driver very careful when there is a child in the car compared to when there is not.

#### **Fatalities by State**

In [19]:
fatalities_by_state = pd.read_sql_query("""
    SELECT
        a.state,
        SUM(a.fatals) AS total_fatalities
    FROM nhtsa.accidents AS a
    WHERE a.state IS NOT NULL
    GROUP BY a.state
    ORDER BY total_fatalities DESC;
""", engine)

In [20]:
fatalities_by_state

,state,total_fatalities
0,California,4428.0
1,Texas,4408.0
2,Florida,3530.0
3,Georgia,1797.0
4,North Carolina,1630.0
5,Tennessee,1314.0
6,Arizona,1302.0
7,Ohio,1275.0
8,Illinois,1268.0
9,Pennsylvania,1179.0


<img src="Assets/Total Number of Fatalities by State.png" alt="drawing" width="1000"/>

<p>.</p>

Most of the fatalities in the USA are in these three states California, Texas, and Florida. These states make 29.09% of the road fatalities. Their total of fatalities were 4,428, 4,408, and 3,530, which makes the state of California as the leading state.
The District of Columbia showed the lowest number of deaths in the year 2022, with only 32 people who died on the road. The state is followed by the Rhode Island and Vermont states, with 52 and 76 deaths respectively.

<img src="Assets/Total Road Fatalities by Population Size.png" alt="drawing" width="1000"/>

<p>.</p>

It can also been seen in the graph above that there is a strong correlation between the total population of a state and the total road fatalities. States with high density population tend to experience a lot of road traffic deaths. The leading states are again California, Texas, and Florida. These states have a high population density, having 39, 30, and 22 million people respectively. These three states represent 27.40% of the USA total population.

### **3.c Major Causes of Road Traffic Fatalities in the USA in 2022**

#### **Fatalities by Distractions**

<img src="Assets/Total Number of Fatalities by Distractions.png" alt="drawing" width="1000"/>

<p>.</p>

Distractions while driving account for a large portion of road fatalities. Inattentive driving stands out as the most significant contributor, with a staggering 1,388 fatalities. This issue often stems from drivers losing focus on the road, whether due to fatigue, zoning out, or simply not paying enough attention. Closely following, distracted or careless driving claims 664 lives, illustrating how critical it is for drivers to maintain full attention on the task at hand.

External distractions, such as being distracted by other people, objects, or events outside the vehicle, cause 289 fatalities. These distractions can range from roadside billboards to accidents on the other side of the highway. The use of mobile phones while driving is another major culprit, with 157 fatalities resulting from manipulating mobile phones and 131 from other mobile phone-related distractions. Even conversations with passengers can lead to fatal outcomes, with 131 fatalities attributed to distractions caused by other occupants in the vehicle.

Using other vehicle components or controls integral to the vehicle leads to 133 fatalities, highlighting the risks involved in adjusting settings while driving. Additionally, activities such as eating or drinking, which may seem harmless, result in 49 fatalities. Adjusting audio or climate controls, lost in thought or daydreaming, objects moving inside the vehicle, and smoking are other distractions that contribute to the death toll, albeit in smaller numbers.

<img src="Assets/Total Fatalities by Violations.png" alt="drawing" width="1000"/>

<p>.</p>

Various traffic violations also contribute heavily to road fatalities. Manslaughter or homicide leads to 1,545 deaths, often resulting from extreme cases of reckless or aggressive driving. Driving while intoxicated, whether by alcohol or drugs, results in 1,359 fatalities, reinforcing the lethal combination of impaired driving and substance abuse.

Failure to yield, a common traffic violation, causes 647 fatalities. This often happens when drivers do not give the right of way to other vehicles or pedestrians, leading to severe collisions. Inattentive, careless, or improper driving accounts for 612 fatalities, further highlighting the importance of maintaining focus and following traffic rules.

Other driver license violations, including driving without a valid license, lead to 574 fatalities. Hit-and-run incidents, where drivers flee the scene of an accident, result in 498 fatalities. Such actions not only cause deaths but also reflect severe legal and ethical breaches. Willful reckless driving, including actions that endanger others, results in 478 fatalities, while serious violations resulting in death account for 468 fatalities.

Lane violations, such as improper lane changes or not staying within designated lanes, cause 417 fatalities. Driving uninsured vehicles leads to 398 fatalities, emphasizing the necessity of legal compliance and responsible vehicle ownership.

<img src="Assets/Top Drugs Involved in Accidents.png" alt="drawing" width="1000"/>

<p>.</p>

Drug use significantly exacerbates the risk of road accidents. Delta 9-tetrahydrocannabinol (THC), the psychoactive component of cannabis, is the most common drug involved in accidents, with 4,217 cases. This is followed by 11-nor-9-carboxy-delta-9-tetrahydrocannabinol (Carboxy THC), another cannabis metabolite, present in 3,633 cases. The prevalence of cannabis-related substances underscores the impact of marijuana on driving safety.

Stimulants like amphetamines and methamphetamines are also major contributors, involved in 2,923 and 2,918 cases, respectively. These drugs can cause erratic behavior and impaired judgment, leading to dangerous driving conditions. The presence of Hydroxy-THC in 1,984 cases and unknown cannabinoids in 1,569 cases further emphasizes the widespread use of cannabis-related substances.

Surprisingly, caffeine, typically consumed to enhance alertness, is involved in 1,564 cases. This could be due to excessive consumption leading to jitteriness or other adverse effects. Fentanyl, a powerful opioid, is present in 1,412 cases, highlighting the dangers of driving under the influence of strong painkillers. Other drugs like benzoylecgonine, a cocaine metabolite, and cocaine itself are involved in 1,222 and 1,131 cases, respectively, indicating the significant role of illicit drugs in road accidents.

<img src="Assets/Percentage of Fatalities by Speeding.png" alt="drawing" width="1000"/>

<p>.</p>

<img src="Assets/Percentage of Fatalities by Impairments.png" alt="drawing" width="1000"/>

<p>.</p>